# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [1]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

from __future__ import print_function

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside **cs231n/classifiers/softmax.py**. 


In [3]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 0.000000
sanity check: 2.302585


In [4]:
X = X_dev
y = y_dev
W = np.random.randn(3073, 10) * 0.0001
reg = 0.0

In [5]:
wx = X.dot(W)
p_unnorm = np.exp(wx-wx.max(1)[:,None])
p_sum = np.sum(p_unnorm,1)[:,None]
p = p_unnorm/p_sum
yp = np.choose(y, p.T)
p_log = np.log(yp)
L = -np.mean(p_log)

In [6]:
print(L)

2.342213673601188


In [7]:
import torch

In [8]:
X = X_dev
y = y_dev
W = np.random.randn(3073, 10) * 0.0001

W = torch.from_numpy(W)
X = torch.from_numpy(X)
W.requires_grad = True
wx = torch.mm(X,W)
#p_unnorm = torch.exp(wx-wx.max(1)[0][:,None])
p_unnorm = torch.exp(wx)
p_sum = torch.sum(p_unnorm,1)[:,None]
p_sum_inv = 1/p_sum
p = p_unnorm * p_sum_inv
yp = p.gather(1,torch.from_numpy(y).long().view(-1,1))
p_log = torch.log(yp)
L = -torch.mean(p_log)

In [9]:
wx.retain_grad()
p_unnorm.retain_grad()
p_sum.retain_grad()
p_sum_inv.retain_grad()
p.retain_grad()
yp.retain_grad()
p_log.retain_grad()
L.retain_grad()

In [10]:
L.backward()

In [11]:
dL = 1
dp_log = torch.zeros_like(p_log)
dp_log[:] = -1/len(dp_log)
dyp = 1/yp
dp = torch.zeros_like(p)
idx = torch.LongTensor(y)
j = torch.arange(dp.size(0)).long()
dp[j, idx] = 1
dp_sum_inv = p_unnorm
dp_unnorm = p_sum_inv
dp_sum = -p_sum**-2
dp_unnorm2 = torch.ones(p_unnorm.shape, dtype=torch.float64)
dwx = torch.exp(wx)
dw = X

In [12]:
dLp_log = dL * dp_log
dLyp = dLp_log * dyp
dLp = dLyp * dp
dLp_sum_inv = (dLp * dp_sum_inv).sum(1)
dLp_unnorm = dLp * dp_unnorm
dLp_sum = dLp_sum_inv[:,None] * dp_sum
dLp_unnorm2 = dLp_sum * dp_unnorm2
dLp_unnorm += dLp_unnorm2
dLwx = dLp_unnorm * dwx
dLw = torch.mm(dw.t(),dLwx)

In [13]:
dLw[0]

tensor([-0.1902, -2.3341,  0.8166,  0.8112,  3.3421,  4.5179,  0.6947,
         1.0169, -3.5749, -5.1002], dtype=torch.float64)

In [14]:
W.grad[0]

tensor([-0.1902, -2.3341,  0.8166,  0.8112,  3.3421,  4.5179,  0.6947,
         1.0169, -3.5749, -5.1002], dtype=torch.float64)

In [15]:
p.grad[0]

tensor(1.00000e-02 *
       [ 0.0000,  0.0000,  0.0000,  0.0000, -2.6743,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], dtype=torch.float64)

In [17]:
p_sum.shape

torch.Size([500, 1])

In [18]:
p_unnorm.grad[0]

tensor(1.00000e-03 *
       [ 0.1846,  0.1846,  0.1846,  0.1846, -2.2837,  0.1846,  0.1846,
         0.1846,  0.1846,  0.1846], dtype=torch.float64)

In [20]:
W.grad[0]

tensor([-0.1902, -2.3341,  0.8166,  0.8112,  3.3421,  4.5179,  0.6947,
         1.0169, -3.5749, -5.1002], dtype=torch.float64)

In [ ]:
dLp_log = dL * dp_logyp

In [154]:
a = torch.tensor([2.0,3.0,4.0])

In [155]:
a

tensor([ 2.,  3.,  4.])

In [156]:
a.requires_grad=True

In [157]:
b = torch.mean(a)

In [158]:
b

tensor(3.)

In [159]:
a.grad

In [160]:
b.backward()

In [161]:
a

tensor([ 2.,  3.,  4.])

In [162]:
a.grad

tensor([ 0.3333,  0.3333,  0.3333])

In [12]:
dL = 1
dLloss_all = np.zeros_like(loss_all)
dLloss_all[:] = 1/len(loss_all)
dLprob_log = dLloss_all * -1
dLprob = dLprob_log * 1/y_prob
dLprob_sum = dLprob[:,None]*(-prob_unnorm/prob_sum[:,None]**2)

In [13]:
prob_sum[:,None]

array([[[5.93444808]],

       [[4.76689704]],

       [[4.56388675]],

       [[4.10764927]],

       [[5.90275043]],

       [[7.58192426]],

       [[3.40302217]],

       [[5.27765457]],

       [[7.19081336]],

       [[6.79689705]],

       [[5.66899338]],

       [[6.75456876]],

       [[6.50630839]],

       [[7.53220523]],

       [[8.76845718]],

       [[3.88084186]],

       [[5.69123946]],

       [[6.83252271]],

       [[8.55499846]],

       [[7.0666757 ]],

       [[4.31694716]],

       [[7.45497268]],

       [[7.91913461]],

       [[6.5734701 ]],

       [[4.95589954]],

       [[4.11548836]],

       [[7.77449496]],

       [[4.14415066]],

       [[4.50180202]],

       [[3.63564524]],

       [[6.84682941]],

       [[8.1114156 ]],

       [[5.90188419]],

       [[5.20497062]],

       [[6.52371026]],

       [[6.16301794]],

       [[5.38436165]],

       [[4.682236  ]],

       [[6.59359848]],

       [[7.47934087]],

       [[4.42039849]],

       [[6.08341

In [94]:
prob_sum.shape

(500, 1)

In [67]:
dLprob_log

array([-0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
       -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002, -0.002,
      

In [51]:
import torch

In [52]:
x = torch.from_numpy(np.array([1.0,2.0,3.0]))

In [53]:
x.requires_grad = True

In [54]:
y = torch.mean(x)

In [55]:
y.backward()

In [56]:
x.grad

tensor([ 0.3333,  0.3333,  0.3333], dtype=torch.float64)

In [18]:
def softmax_loss_naive(W, X, y, reg):
  """
  Softmax loss function, naive implementation (with loops)

  Inputs have dimension D, there are C classes, and we operate on minibatches
  of N examples.

  Inputs:
  - W: A numpy array of shape (D, C) containing weights.
  - X: A numpy array of shape (N, D) containing a minibatch of data.
  - y: A numpy array of shape (N,) containing training labels; y[i] = c means
    that X[i] has label c, where 0 <= c < C.
  - reg: (float) regularization strength

  Returns a tuple of:
  - loss as single float
  - gradient with respect to weights W; an array of same shape as W
  """
  # Initialize the loss and gradient to zero.
  loss = 0.0
  dW = np.zeros_like(W)

  #############################################################################
  # TODO: Compute the softmax loss and its gradient using explicit loops.     #
  # Store the loss in loss and the gradient in dW. If you are not careful     #
  # here, it is easy to run into numeric instability. Don't forget the        #
  # regularization!                                                           #
  #############################################################################
  
  #############################################################################
  #                          END OF YOUR CODE                                 #
  #############################################################################

  return loss, dW

## Inline Question 1:
Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

**Your answer:** *Fill this in*


In [24]:
grad

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, relative error: nan
numerical: 0.000000 analytic: 0.000000, 

C:\work\code\repos\pytorch\assignment\assignment1\cs231n\gradient_check.py:124: RuntimeWarning: invalid value encountered in double_scalars
  rel_error = abs(grad_numerical - grad_analytic) / (abs(grad_numerical) + abs(grad_analytic))


In [ ]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.
from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################
# Your code
################################################################################
#                              END OF YOUR CODE                                #
################################################################################
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question** - *True or False*

It's possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

*Your answer*:

*Your explanation*:

In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])